In [1]:
import pandas as pd
import numpy as np
import pysam

#### 1. Read gff3 file into a pandas dataframe and examine the GENCODE file format

In [2]:
gencode = pd.read_table("/Users/pubudu/Documents/RefData/Gencode/gencode.v19.annotation.gff3", comment="#",
                        sep = "\t", names = ['seqname', 'source', 'feature', 'start' , 'end', 'score', 'strand', 'frame', 'attribute'])
gencode.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,11869,14412,.,+,.,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000223972.4;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...


In [3]:
gencode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615566 entries, 0 to 2615565
Data columns (total 9 columns):
seqname      object
source       object
feature      object
start        int64
end          int64
score        object
strand       object
frame        object
attribute    object
dtypes: int64(2), object(7)
memory usage: 179.6+ MB


#### 2. Extract Genes in the gff3 file "feature = gene"

In [4]:
gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'start', 'end', 'attribute']].copy().reset_index().drop('index', axis=1) # Extract genes
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57820 entries, 0 to 57819
Data columns (total 4 columns):
seqname      57820 non-null object
start        57820 non-null int64
end          57820 non-null int64
attribute    57820 non-null object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


In [5]:
gencode_genes.head()

,seqname,start,end,attribute
0,chr1,11869,14412,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...
1,chr1,14363,29806,ID=ENSG00000227232.4;gene_id=ENSG00000227232.4...
2,chr1,29554,31109,ID=ENSG00000243485.2;gene_id=ENSG00000243485.2...
3,chr1,34554,36081,ID=ENSG00000237613.2;gene_id=ENSG00000237613.2...
4,chr1,52473,54936,ID=ENSG00000268020.2;gene_id=ENSG00000268020.2...


#### 3. Extract gene_name, gene_type, gene_status, level of each gene

In [6]:
def gene_info(x):
    # Extract gene names
    g_name = list(filter(lambda x: 'gene_name' in x,  x.split(";")))[0].split("=")[1]
    g_type = list(filter(lambda x: 'gene_type' in x,  x.split(";")))[0].split("=")[1]
    g_status = list(filter(lambda x: 'gene_status' in x,  x.split(";")))[0].split("=")[1]
    g_leve = int(list(filter(lambda x: 'level' in x,  x.split(";")))[0].split("=")[1])
    return (g_name, g_type, g_status, g_leve)

In [7]:
gencode_genes["gene_name"], gencode_genes["gene_type"], gencode_genes["gene_status"], gencode_genes["gene_level"] = zip(*gencode_genes.attribute.apply(lambda x: gene_info(x)))
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_status,gene_level
0,chr1,11869,14412,ID=ENSG00000223972.4;gene_id=ENSG00000223972.4...,DDX11L1,pseudogene,KNOWN,2
1,chr1,14363,29806,ID=ENSG00000227232.4;gene_id=ENSG00000227232.4...,WASH7P,pseudogene,KNOWN,2
2,chr1,29554,31109,ID=ENSG00000243485.2;gene_id=ENSG00000243485.2...,MIR1302-11,lincRNA,NOVEL,2
3,chr1,34554,36081,ID=ENSG00000237613.2;gene_id=ENSG00000237613.2...,FAM138A,lincRNA,KNOWN,2
4,chr1,52473,54936,ID=ENSG00000268020.2;gene_id=ENSG00000268020.2...,OR4G4P,pseudogene,KNOWN,2


In [8]:
gencode_genes['gene_type'].drop_duplicates()

0                      pseudogene
2                         lincRNA
6                  protein_coding
13                      antisense
14           processed_transcript
15                          snRNA
76                 sense_intronic
82                          miRNA
110                      misc_RNA
254                        snoRNA
315                          rRNA
608      3prime_overlapping_ncrna
614        polymorphic_pseudogene
1274            sense_overlapping
6806                    IG_V_gene
6849                    IG_C_gene
6850                    IG_J_gene
6857              IG_V_pseudogene
21480                   TR_C_gene
21481                   TR_J_gene
21487                   TR_V_gene
21488             TR_V_pseudogene
26189             IG_C_pseudogene
38188                   TR_D_gene
38198             TR_J_pseudogene
39997             IG_J_pseudogene
40004                   IG_D_gene
57783                     Mt_tRNA
57784                     Mt_rRNA
Name: gene_typ

#### 4. Extract all known protein_coding genes

In [9]:
gencode_genes = gencode_genes[gencode_genes['gene_status'] == 'KNOWN'].reset_index().drop('index', axis=1)
gencode_genes = gencode_genes[gencode_genes['gene_type'] == 'protein_coding'].reset_index().drop('index', axis=1)
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19456 entries, 0 to 19455
Data columns (total 8 columns):
seqname        19456 non-null object
start          19456 non-null int64
end            19456 non-null int64
attribute      19456 non-null object
gene_name      19456 non-null object
gene_type      19456 non-null object
gene_status    19456 non-null object
gene_level     19456 non-null int64
dtypes: int64(3), object(5)
memory usage: 1.2+ MB


#### 5. Remove duplicates - Prioritize verified and manually annotated loci over automatically annotated loci
Gene level column can be used to prioritize genes when removing duplicates 
1. verified loci
2. manually annotated loci
3. automatically annotated loci

In [10]:
## Sort gene_leve in each chromosome (ascending oder) and remove duplicates
gencode_genes = gencode_genes.sort_values(['gene_level', 'seqname'], ascending=True).drop_duplicates('gene_name', keep='first').reset_index().drop('index', axis=1) 
gencode_genes.head()

,seqname,start,end,attribute,gene_name,gene_type,gene_status,gene_level
0,chr1,1246965,1260071,ID=ENSG00000127054.14;gene_id=ENSG00000127054....,CPSF3L,protein_coding,KNOWN,1
1,chr1,1337288,1342693,ID=ENSG00000242485.1;gene_id=ENSG00000242485.1...,MRPL20,protein_coding,KNOWN,1
2,chr1,1353800,1357149,ID=ENSG00000235098.4;gene_id=ENSG00000235098.4...,ANKRD65,protein_coding,KNOWN,1
3,chr1,1550795,1565990,ID=ENSG00000197530.8;gene_id=ENSG00000197530.8...,MIB2,protein_coding,KNOWN,1
4,chr1,1716729,1822495,ID=ENSG00000078369.13;gene_id=ENSG00000078369....,GNB1,protein_coding,KNOWN,1


In [11]:
gencode_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19410 entries, 0 to 19409
Data columns (total 8 columns):
seqname        19410 non-null object
start          19410 non-null int64
end            19410 non-null int64
attribute      19410 non-null object
gene_name      19410 non-null object
gene_type      19410 non-null object
gene_status    19410 non-null object
gene_level     19410 non-null int64
dtypes: int64(3), object(5)
memory usage: 1.2+ MB


* Step 5 removed 46 records (19455-19409)

In [12]:
gencode_genes.to_csv('gencode.v19.annotation.gff3_all_known_genes.txt', index=False, header = False, sep="\t")

In [13]:
%%bash -s gencode.v19.annotation.gff3_all_known_genes.txt
cut -f 1,2,3,5 $1 | sortBed -i > gencode.v19.annotation.gff3_all_known_genes.txt.sorted.formatted.txt